# Common functionality

In [ ]:
from time import sleep
import apsuite.commisslib.meas_fofb_sysid as fofbsysid
print(fofbsysid.__file__)
from multiprocessing import Pool, set_start_method
from resource import getrusage, RUSAGE_SELF

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import matplotlib
import numpy as np
import h5py

from apsuite.commisslib.meas_fofb_sysid import FOFBSysIdAcq
from siriuspy.devices import EVG, InjCtrl, SOFB, Trigger
%matplotlib widget

In [ ]:
fpath = '/ibira/lnls/labs/swc/MachineStudies/teste/2024-03-25/bpm-lfsr9-sd4-rfprofile/'

In [ ]:
acq = FOFBSysIdAcq()

# Data analysis

In [ ]:
fft = lambda x: np.abs(np.fft.rfft(x))

def init_acq_from_data(acq, data):
    # parameters to generate singular mode levels
    params = data['params']
    acq.params.svd_levels_regularize_matrix = True
    acq.params.svd_levels_reg_sinval_min = data['params']['svd_levels_reg_sinval_min']
    acq.params.svd_levels_reg_tikhonov_const = data['params']['svd_levels_reg_tikhonov_const']
    acq.params.svd_levels_bpmsx_enbllist = data['params']['svd_levels_bpmsx_enbllist']
    acq.params.svd_levels_bpmsy_enbllist = data['params']['svd_levels_bpmsy_enbllist']
    acq.params.svd_levels_ch_enbllist = data['params']['svd_levels_ch_enbllist']
    acq.params.svd_levels_cv_enbllist = data['params']['svd_levels_cv_enbllist']
    acq.params.svd_levels_rf_enbllist = data['params']['svd_levels_rf_enbllist']
    acq.params.svd_levels_respmat = data['params']['svd_levels_respmat']

    acq.params.svd_levels_singmode_idx = data['params']['svd_levels_singmode_idx']

## Plot all sensitivity functions at once

In [ ]:
def get_mode_fft(file, acq=None):
    
    if acq is None:
        acq = FOFBSysIdAcq()
    
    print(f'reading {file}...')
    data = acq.load_data(file)
    init_acq_from_data(acq, data)
    
    #================ INPUTS ================#
    rf_acq = False
    factor = 100_000
    val = 1000
    #========================================#
    
    if rf_acq:
        prbs = np.array(data['data']['prbs_data'][0], dtype=float)
        prbs[prbs == 0] = -1
    
        acq.params.svd_levels_singmode_idx  = 0
        acq.params.prbs_corrs_to_get_data = np.ones(160, dtype=bool)
        
        rfprof = data['data']['fofb_respmat'][:,-1].copy()
        rfprof /= np.linalg.norm(rfprof)
        rfprof *= val * factor
        rfprof_x = rfprof[:160]
        rfprof_y = rfprof[160:]
        
        lvlsx = rfprof_x
        lvlsy = rfprof_y
    
        u_n = np.hstack([lvlsx[acq.params.svd_levels_bpmsx_enbllist], lvlsy[acq.params.svd_levels_bpmsy_enbllist]])
        u_n = np.array([u_n], dtype=float)
        u_n /= factor
        
    else:
        
        prbs = np.array(data['data']['prbs_data'][0], dtype = float)
        prbs[prbs == 0] = -1
        
        acq.params.svd_levels_singmode_idx = data['params']['svd_levels_singmode_idx']
        lvlsx, lvlsy = acq.get_levels_bpms_from_svd(ampmax = acq.params.svd_levels_ampmax*factor, lvl = -val*factor)
        u_n = np.hstack([lvlsx[acq.params.svd_levels_bpmsx_enbllist], lvlsy[acq.params.svd_levels_bpmsy_enbllist]])
        u_n = np.array([u_n], dtype=float)
        u_n /= factor

    prbsu = np.array([prbs]).T @ u_n

    orb = np.array(np.hstack([data['data']['orbx'], data['data']['orby']]), dtype=float)
    orb -= np.mean(orb, axis=0)[None, :]
    
    prbsu -= np.mean(prbsu, axis=0)[None, :]

    modes_orb = (orb @ u_n.T)[:,0]
    modes_prbs = (prbsu @ u_n.T)[:,0]

    lfsr = data['data']['prbs_lfsr_len'][0]
    sd = data['data']['prbs_step_duration'][0]
    p = (2**lfsr - 1) * sd

    n = modes_orb.shape[0]
    step = n // p

    fft_orb = fft(modes_orb)
    fft_prbs = fft(modes_prbs)
    fs = data['data']['sampling_frequency']
    freq = np.fft.rfftfreq(modes_orb.size, d=1/fs)

    freq_p = freq[step::step]
    fft_prbs_p = fft_prbs[step::step]
    fft_orb_p = fft_orb[step::step]

    return freq_p, fft_orb_p, fft_prbs_p

def calc_sensitivity(freq, orb, prbs):
    outside_notch = prbs > 0
    return freq[outside_notch], orb[outside_notch]/prbs[outside_notch]

In [ ]:
# change processes to account for memory availability. each process takes about 3GB
n_modes = 100
paths = [fpath + f'{i}.h5' for i in range(n_modes)]
%time sensitivities = Pool(processes=32).map(get_mode_fft, paths)

In [ ]:
valid_modes = 100

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,8))
for i, (f, orb, prbs) in enumerate(sensitivities[:valid_modes]):
    f, s = calc_sensitivity(f, orb, prbs)
    interest = np.logical_and(f > 1e3, f<10e3)
    interest = f < 10e3
    ax.semilogx(f[interest], 20*np.log10(np.abs(s[interest])), label=f'{i}')
matplotlib.rcParams.update({'font.size': 16})
ax.set_title(f'SIRIUS FOFB Disturbance Rejection - {valid_modes} modes')
ax.set_ylabel('dB')
ax.set_xlabel('Hz')+
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.grid(which='both')

fig.savefig('rfprofile_semLinhadeRF_ki30kp3.png', dpi=1200)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12,8))
for i, (f, orb, prbs) in enumerate(sensitivities[:valid_modes]):
    f, s = calc_sensitivity(f, orb, prbs)
    interest = np.logical_and(f > 1e3, f<10e3)
    ax.semilogx(f[interest], 20*np.log10(np.abs(s[interest])), label=f'{i}')

matplotlib.rcParams.update({'font.size': 16})
ax.set_title(f'SIRIUS FOFB Disturbance Rejection - {valid} modes')
ax.set_ylabel('dB')
ax.set_xlabel('Hz')
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.grid(which='both')

fig.savefig('gain0p1901kHz.png', dpi=1200)

## Plot data from two acquisitions at once

In [ ]:
freq_p_old = freq_p
fft_prbs_p_old = fft_prbs_p
fft_orb_p_old = fft_orb_p

In [ ]:
fig, (ax, ay) = plt.subplots(2)
ax.loglog(freq_p, fft_prbs_p)
ax.loglog(freq_p, fft_orb_p)
ax.loglog(freq_p_old, fft_prbs_p_old)
ax.loglog(freq_p_old, fft_orb_p_old)

ay.loglog(freq_p, (fft_orb_p/fft_prbs_p), '--')
ay.loglog(freq_p_old, (fft_orb_p_old/fft_prbs_p_old), '--', alpha=.5)
ay.grid()

In [ ]:
fft_orb_p/fft_prbs_p

In [ ]:
data = acq.get_data()

In [ ]:
[all(data['prbs_data'][0] == data['prbs_data'][i]) for i in range(20)]